In [52]:
from typing import Any, Dict, Union
import pandas as pd
import logging
import re
import sqlfluff
import requests
import ruamel.yaml
import psycopg2
from bs4 import BeautifulSoup
from markdown import markdown
import datetime
from urllib.parse import unquote
from typing import Any, Dict, Iterator, List, Union
import json
from pgsanity.pgsanity import check_string
import os
from dbt.cli.main import dbtRunner, dbtRunnerResult
import smtplib
import ssl
from itertools import compress
import numpy as np
from pathlib import Path
from jinja2 import Template
with open(DBT_PROJECT_DIR + "create_model.txt", "r") as f:
    MODEL_TEMPLATE = Template(f.read())

In [99]:
MATERIALIZATION_MAPPING = {1: "table", 2: "view", 3: "incremental", 4: "ephemereal"}
SUPERSET_USERNAME = "fdp"
SUPERSET_PASSWORD = "fdp"
SUPERSET_HOST = "http://34.82.185.252:30007/"
DATABASE_USERNAME = "fdp"
DATABASE_PASSWORD = "fdp"
DATABASE_HOST = "34.82.185.252"
DATABASE_PORT = 30007
DATABASE_NAME = "financial_data"
QUERY_SCHEMA="financial_query"
QUERY_TABLE="query"
MANIFEST_PATH="/home/vu/Desktop/Projects/Thesis/financial/dbt/target/manifest.json"
EMAIL_PORT = 465
SMTP = "smtp.gmail.com"
EMAIL_SENDER = "catvu113@gmail.com"
EMAIL_PASSWORD = "xhtzakhmnsbufufy"
USER_MODEL_PATH = "/home/vu/Desktop/Projects/Thesis/financial/dbt/models/user"
DBT_PROJECT_DIR = "/home/vu/Desktop/Projects/Thesis/financial/dbt/"
DATABASE_ID = 1
SUPERSET_ID = 36
USER_SCHEMA = "user"
SERVING_SCHEMA="marts"
context = ssl.create_default_context()
SMTP = smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context)
SST_DATABASE_NAME = "FDP Reader"
SERVING_AND_USER_WITH_DOT = (SERVING_SCHEMA+'.',USER_SCHEMA+'.')

In [5]:
from dagster_graphql import DagsterGraphQLClient
client = DagsterGraphQLClient("34.82.185.252", port_number=30003)
status = client.reload_repository_location("financial")

/tmp/ipykernel_187818/1731185254.py:2: ExperimentalWarning: "DagsterGraphQLClient" is an experimental class. It may break in future versions, even between dot releases. To mute warnings for experimental functionality, invoke warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning) or use one of the other methods described at https://docs.python.org/3/library/warnings.html#describing-warning-filters.
  client = DagsterGraphQLClient("34.82.185.252", port_number=30003)


In [141]:
class SupersetDBTSessionConnector:
    """A class for accessing the Superset API in an easy way."""

    def __init__(self):
        """Instantiates the class.

        ''access_token'' will be instantiated via enviromental variable
        If ``access_token`` is None, attempts to obtain it using ``refresh_token``.

        Args:
            api_url: Base API URL of a Superset instance, e.g. https://my-superset/api/v1.
            access_token: Access token to use for accessing protected endpoints of the Superset
                API. Can be automatically obtained if ``refresh_token`` is not None.
            refresh_token: Refresh token to use for obtaining or refreshing the ``access_token``.
                If None, no refresh will be done.
        """
        self.url = SUPERSET_HOST
        self.api_url = self.url + "api/v1/"

        self.session = requests.session()

        self.username = SUPERSET_USERNAME
        self.password = SUPERSET_PASSWORD

        self.refresh_session()

    def refresh_session(self):
        logging.info("Refreshing session")

        soup = BeautifulSoup(self.session.post(self.url + "login").text, "html.parser")
        self.csrf_token = soup.find("input", {"id": "csrf_token"})["value"]  # type: ignore

        data = {
            "username": self.username,
            "password": self.password,
            "provider": "db",
            "refresh": True,
        }
        headers = {
            # 'Authorization': 'Bearer {}'.format(self.____access_token),
            "x-csrftoken": self.csrf_token,
        }
        response = self.session.post(self.url + "login", json=data, headers=headers)  # type: ignore
        return True

    def request(self, method, endpoint, **request_kwargs):
        """Executes a request against the Superset API.

        Args:
            method: HTTP method to use.
            endpoint: Endpoint to use.
            **request_kwargs: Any ``requests.request`` arguments to use.

        Returns:
            A dictionary containing response body parsed from JSON.

        Raises:
            HTTPError: There is an HTTP error (detected by ``requests.Response.raise_for_status``)
                even after retrying with a fresh session.
        """

        logging.info("About to %s execute request for endpoint %s", method, endpoint)

        url = self.api_url + endpoint
        csrf_headers = {
            # 'Authorization': 'Bearer {}'.format(self.__access_token),
            "x-csrftoken": self.csrf_token,
        }

        res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

        logging.info("Request finished with status: %d", res.status_code)

        if res.status_code == 401 and self.refresh_session():
            logging.info(f"Retrying {method} request for {url} %s with refreshed session")
            res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

            logging.info("Request finished with status: %d", res.status_code)

        if res.status_code == 400 and self.refresh_session():
            logging.info(f"Retrying {method} request for {url} %s with refreshed session")
            res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore
            logging.info(f"Request finished with status: {res.status_code}")
        res.raise_for_status()
        return res.json()


def get_tables_from_dbt(dbt_manifest, dbt_db_name):
    tables = {}
    for table_type in ["nodes"]:
        manifest_subset = dbt_manifest[table_type]

        for table_key_long in manifest_subset:
            table = manifest_subset[table_key_long]
            name = table["name"]
            schema = table["schema"]
            database = table["database"]
            description = table["description"]
            alias = table["alias"]
            source = table["unique_id"].split(".")[-2]
            table_key = schema + "." + alias  # Key will be alias, not name
            columns = table["columns"]

            if dbt_db_name is None or database == dbt_db_name:
                # fail if it breaks uniqueness constraint
                assert table_key not in tables, (
                    f"Table {table_key} is a duplicate name (schema + table) across databases. "
                    "This would result in incorrect matching between Superset and dbt. "
                    "To fix this, remove duplicates or add ``dbt_db_name``."
                )
                tables[table_key] = {
                    "name": name,
                    "schema": schema,
                    "database": database,
                    "type": table_type[:-1],
                    "ref": f"ref('{name}')" if table_type == "nodes" else f"source('{source}', '{name}')",
                    "user": None,
                    "columns": columns,
                    "description": description,
                    "alias": alias,
                }
            if schema == "user":
                tables[table_key]["user"] = table["tags"][0]

    assert tables, "Manifest is empty!"

    return tables


def get_physical_datasets_from_superset(superset: SupersetDBTSessionConnector, superset_db_id):
    logging.info("Getting physical datasets from Superset.")
    page_number = 0
    datasets = []
    datasets_keys = set()
    while True:
        logging.info("Getting page %d.", page_number + 1)
        rison_request = f"dataset/?q=(page_size:100,page:{page_number},order_column:changed_on_delta_humanized,order_direction:asc,filters:!((col:table_name,opr:nct,value:archived),(col:sql,opr:dataset_is_null_or_empty,value:true)))"
        res = superset.request("GET", rison_request)
        result = res["result"]
        if result:
            for r in result:
                name = r["table_name"]
                schema = r["schema"]
                database_name = r["database"]["database_name"]
                dataset_id = r["id"]
                database_id = r["database"]["id"]
                dataset_key = f"{schema}.{name}"  # same format as in dashboards

                kind = r["kind"]
                if kind == "physical" and (superset_db_id is None or database_id == superset_db_id):
                    dataset_id = r["id"]

                    name = r["table_name"]
                    schema = r["schema"]
                    dataset_key = f"{schema}.{name}"  # used as unique identifier

                    dataset_dict = {
                        "id": dataset_id,
                        "name": name,
                        "schema": schema,
                        "database": database_name,
                        "dataset_id": dataset_id,
                        "key": dataset_key,
                        "table": [dataset_key],
                    }

                    # fail if it breaks uniqueness constraint
                    assert dataset_key not in datasets_keys, (
                        f"Dataset {dataset_key} is a duplicate name (schema + table) "
                        "across databases. "
                        "This would result in incorrect matching between Superset and dbt. "
                        "To fix this, remove duplicates or add the ``superset_db_id`` argument."
                    )

                    datasets_keys.add(dataset_key)
                    datasets.append(dataset_dict)

            page_number += 1
        else:
            break

    return datasets


def get_tables_from_sql_simple(sql):
    """
    (Superset) Fallback SQL parsing using regular expressions to get tables names, ignoring tables without schema.
    """
    sql = re.sub(r"(--.*)|(#.*)", "", sql)
    sql = re.sub(r"\s+", " ", sql)
    sql = re.sub(r"(/\*(.|\n)*\*/)", "", sql)

    regex = re.compile(r"\b(from|join)\b\s+(\"?(\w+)\"?(\.))?\"?(\w+)\"?\b")
    tables_match = regex.findall(sql)
    tables = [
        table[2] + "." + table[4] if table[2] != "" else table[4] for table in tables_match if table[4] != "unnest"
    ]

    tables = list(set(tables))
    final_tables = [table for table in tables if table.startswith(SERVING_AND_USER_WITH_DOT)] # Assuming that no schema reference
    return tables



def get_tables_from_sql(sql, dialect, sql_parsed=None):
    """
    (Superset) SQL parsing using sqlfluff to get clean tables names.
    If sqlfluff parsing fails it runs the above regex parsing func.
    Returns a tables list.
    """
    try:
        if not sql_parsed:
            sql_parsed = sqlfluff.parse(sql, dialect=dialect)
        tables_raw = list(get_json_segment(sql_parsed, "table_reference"))  # type: ignore
        tables_cleaned = []  # With schema
        for table_ref in tables_raw:
            if isinstance(table_ref, list):
                table_ref_identifier = []
                # Get last 2 "naked_identifier"
                for dictionary in table_ref[::-1]:
                    if "naked_identifier" in dictionary:
                        table_ref_identifier.append(dictionary["naked_identifier"])
                        if len(table_ref_identifier) == 2:
                            tables_cleaned.append(".".join(table_ref_identifier[::-1]))
                            break
            if isinstance(table_ref, dict):
                tables_cleaned.append(table_ref["naked_identifier"])
    except (
        sqlfluff.core.errors.SQLParseError,  # type: ignore
        sqlfluff.core.errors.SQLLexError,  # type: ignore
        sqlfluff.core.errors.SQLFluffUserError,  # type: ignore
        sqlfluff.api.simple.APIParsingError,  # type: ignore
    ) as e:  # type: ignore
        logging.warning(
            "Parsing SQL through sqlfluff failed. "
            "Let me attempt this via regular expressions at least and "
            "check the problematic query and error below.\n%s",
            sql,
            exc_info=e,
        )
        tables_cleaned = get_tables_from_sql_simple(sql)

    return tables_cleaned


def get_json_segment(
    parse_result: Dict[str, Any], segment_type: str
) -> Iterator[Union[str, Dict[str, Any], List[Dict[str, Any]]]]:
    """Recursively search JSON parse result for specified segment type.

    Args:
        parse_result (Dict[str, Any]): JSON parse result from `sqlfluff.fix`.
        segment_type (str): The segment type to search for.

    Yields:
        Iterator[Union[str, Dict[str, Any], List[Dict[str, Any]]]]:
        Retrieves children of specified segment type as either a string for a raw
        segment or as JSON or an array of JSON for non-raw segments.
    """
    for k, v in parse_result.items():
        if k == segment_type:
            yield v
        elif isinstance(v, dict):
            yield from get_json_segment(v, segment_type)
        elif isinstance(v, list):
            for s in v:
                yield from get_json_segment(s, segment_type)


def get_dashboards_from_superset(superset: SupersetDBTSessionConnector, superset_db_id, user_id):
    """
    This function gets
    1. Get dashboards id list from Superset iterating on the pages of the url
    2. Get a dashboard detail information :
        title, owner, url, unique datasets names

    Returns dashboards, dashboards_datasets
    """

    logging.info("Getting published dashboards from Superset.")
    page_number = 0
    dashboards_id = []
    while True:
        logging.info("Getting page %d.", page_number + 1)
        res = superset.request("GET", f'/dashboard/?q={{"page":{page_number},"page_size":100}}')
        result = res["result"]
        if result:
            for r in result:
                if r["published"] and r["created_by"]["id"] == user_id:
                    dashboards_id.append(r["id"])
            page_number += 1
        else:
            break

    assert len(dashboards_id) > 0, "There are no dashboards in Superset!"

    logging.info("There are %d published dashboards in Superset.", len(dashboards_id))

    dashboards = []
    dashboards_datasets_w_db = set()
    for i, d in enumerate(dashboards_id):
        logging.info("Getting info for dashboard %d/%d.", i + 1, len(dashboards_id))
        res = superset.request("GET", f"/dashboard/{d}")
        result = res["result"]

        dashboard_id = result["id"]
        title = result["dashboard_title"]
        url = SUPERSET_PUBLIC_HOST + "superset/dashboard/" + str(dashboard_id)
        owner_name = result["owners"][0]["first_name"] + " " + result["owners"][0]["last_name"]

        # take unique dataset names, formatted as "[database].[schema].[table]" by Superset
        res_table_names = superset.request("GET", f"/dashboard/{d}/datasets")
        result_table_names = res_table_names["result"]

        testing = []
        for i in range(0, len(result_table_names)):
            testing.append(result_table_names[i]["name"])

        # datasets_raw = list(set(result['table_names'].split(', ')))
        datasets_raw = testing

        # parse dataset names into parts
        datasets_parsed = [dataset[1:-1].split("].[", maxsplit=2) for dataset in datasets_raw]
        datasets_parsed = [
            [dataset[0], "None", dataset[1]]  # add None in the middle
            if len(dataset) == 2
            else dataset  # if missing the schema
            for dataset in datasets_parsed
        ]

        # put them all back together to get "database.schema.table"
        datasets_w_db = [".".join(dataset) for dataset in datasets_parsed]

        dbt_project_name = "your_dbt_project."
        datasets_w_db = [dbt_project_name + sub for sub in testing]

        dashboards_datasets_w_db.update(datasets_w_db)

        # skip database, i.e. first item, to get only "schema.table"
        datasets_wo_db = [".".join(dataset[1:]) for dataset in datasets_parsed]

        datasets_wo_db = testing
        dashboard = {
            "id": dashboard_id,
            "title": title,
            "url": url,
            "owner_name": owner_name,
            "datasets": datasets_wo_db,  # add in "schema.table" format
        }
        dashboards.append(dashboard)
    # test if unique when database disregarded
    # loop to get the name of duplicated dataset and work with unique set of datasets w db
    dashboards_datasets = set()
    for dataset_w_db in dashboards_datasets_w_db:
        dataset = ".".join(dataset_w_db.split(".")[1:])  # similar logic as just a bit above

        # fail if it breaks uniqueness constraint and not limited to one database
        assert dataset not in dashboards_datasets or superset_db_id is not None, (
            f"Dataset {dataset} is a duplicate name (schema + table) across databases. "
            "This would result in incorrect matching between Superset and dbt. "
            "To fix this, remove duplicates or add ``superset_db_id``."
        )

        dashboards_datasets.add(dataset)

    return dashboards, dashboards_datasets


def get_datasets_from_superset_dbt_refs(
    superset: SupersetDBTSessionConnector, dashboards_datasets, dbt_tables, sql_dialect, superset_db_id
):
    """
    Returns datasets (dict) containing table info and dbt references
    """

    logging.info("Getting datasets info from Superset.")
    page_number = 0
    datasets = {}
    while True:
        logging.info("Getting page %d.", page_number + 1)
        res = superset.request("GET", f'/dataset/?q={{"page":{page_number},"page_size":100}}')
        result = res["result"]
        if result:
            for r in result:
                name = r["table_name"]
                schema = r["schema"]
                database_name = r["database"]["database_name"]
                database_id = r["database"]["id"]

                dataset_key = f"{schema}.{name}"  # same format as in dashboards

                # only add datasets that are in dashboards, optionally limit to one database
                if dataset_key in dashboards_datasets and (superset_db_id is None or database_id == superset_db_id):
                    kind = r["kind"]
                    if kind == "virtual":  # built on custom sql
                        sql = r["sql"]
                        tables = get_tables_from_sql(sql, sql_dialect)
                        tables = [table if "." in table else f"{schema}.{table}" for table in tables]
                    else:  # built on tables
                        tables = [dataset_key]
                    dbt_refs = [dbt_tables[table]["ref"] for table in tables if table in dbt_tables]

                    datasets[dataset_key] = {
                        "name": name,
                        "schema": schema,
                        "database": database_name,
                        "kind": kind,
                        "tables": tables,
                        "dbt_refs": dbt_refs,
                    }
            page_number += 1
        else:
            break

    return datasets


def refresh_columns_in_superset(superset: SupersetDBTSessionConnector, dataset_id):
    logging.info("Refreshing columns in Superset.")
    superset.request("PUT", f"/dataset/{dataset_id}/refresh")


def add_sst_dataset_metadata(superset: SupersetDBTSessionConnector, dataset_id, sst_dataset_key, dbt_tables):
    logging.info("Refreshing columns in Superset.")
    body = {
        "extra": '{"certification": \n  {"certified_by": "Data Analytics Team", \n   "details": "This table is the source of truth." \n    \n  }\n}',
        "description": dbt_tables[sst_dataset_key]["description"],
        "owners": [SUPERSET_ID],
    }
    if dbt_tables[sst_dataset_key]["user"]:
        body["owners"].append(dbt_tables[sst_dataset_key]["user"])
    superset.request("PUT", f"/dataset/{dataset_id}", json=body)


def add_superset_columns(superset: SupersetDBTSessionConnector, dataset):
    logging.info("Pulling fresh columns info from Superset.")
    res = superset.request("GET", f"/dataset/{dataset['id']}")
    columns = res["result"]["columns"]
    dataset["columns"] = columns
    return dataset


def convert_markdown_to_plain_text(md_string):
    """Converts a markdown string to plaintext.

    The following solution is used:
    https://gist.github.com/lorey/eb15a7f3338f959a78cc3661fbc255fe
    """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(md_string)

    # remove code snippets
    html = re.sub(r"<pre>(.*?)</pre>", " ", html)
    html = re.sub(r"<code>(.*?)</code >", " ", html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = "".join(soup.findAll(text=True))

    # make one line
    single_line = re.sub(r"\s+", " ", text)

    # make fixes
    single_line = re.sub("→", "->", single_line)
    single_line = re.sub("<null>", '"null"', single_line)

    return single_line


def merge_columns_info(dataset, tables):
    logging.info("Merging columns info from Superset and manifest.json file.")

    key = dataset["key"]
    sst_columns = dataset["columns"]
    dbt_columns = tables.get(key, {}).get("columns", {})
    columns_new = []
    for sst_column in sst_columns:
        # add the mandatory field
        column_new = {"column_name": sst_column["column_name"]}

        # add optional fields only if not already None, otherwise it would error out
        for field in [
            "expression",
            "filterable",
            "groupby",
            "python_date_format",
            "verbose_name",
            "type",
            "is_dttm",
            "is_active",
        ]:
            if sst_column[field] is not None:
                column_new[field] = sst_column[field]

        # add description
        if (
            sst_column["column_name"] in dbt_columns
            and "description" in dbt_columns[sst_column["column_name"]]
            and sst_column["expression"] == ""
        ):  # database columns
            description = dbt_columns[sst_column["column_name"]]["description"]
            description = convert_markdown_to_plain_text(description)
        else:  # if cant find in dbt
            description = sst_column["description"]
        column_new["description"] = description

        columns_new.append(column_new)

    dataset["columns_new"] = columns_new

    return dataset


def put_columns_to_superset(superset: SupersetDBTSessionConnector, dataset):
    logging.info("Putting new columns info with descriptions back into Superset.")
    body = {"columns": dataset["columns_new"]}
    superset.request("PUT", f"/dataset/{dataset['id']}?override_columns=true", json=body)


def merge_dashboards_with_datasets(dashboards, datasets):
    for dashboard in dashboards:
        refs = set()
        for dataset in dashboard["datasets"]:
            if dataset in datasets:
                refs.update(datasets[dataset]["dbt_refs"])
        refs = list(sorted(refs))

        dashboard["refs"] = refs

    return dashboards


def get_exposures_dict(dashboards):
    dashboards.sort(key=lambda dashboard: dashboard["id"])
    titles = [dashboard["title"] for dashboard in dashboards]
    # fail if it breaks uniqueness constraint for exposure names
    assert len(set(titles)) == len(titles), "There are duplicate dashboard names!"

    exposures_dict = [
        {
            "name": f"superset__{dashboard['title']}",
            "type": "dashboard",
            "url": dashboard["url"],
            "depends_on": dashboard["refs"],
            "owner": {"name": dashboard["owner_name"]},
        }
        for dashboard in dashboards
    ]

    return exposures_dict


class YamlFormatted(ruamel.yaml.YAML):
    def __init__(self):
        super(YamlFormatted, self).__init__()
        self.default_flow_style = False
        self.allow_unicode = True
        self.encoding = "utf-8"
        self.block_seq_indent = 2
        self.indent = 4
        self.emitter.alt_null = "''"


# Create Query


def is_valid_table_name(table_name):
    """
    Checks if the given string is a valid table name in PostgreSQL.

    Args:
        table_name: The string to check.

    Returns:
        True if the string is a valid table name, False otherwise.
    """

    # The regular expression to match a valid table name.
    regex = re.compile(r"^[a-zA-Z0-9_]{1,63}$")

    # Check if the string matches the regular expression.
    if regex.match(table_name):
        return True
    else:
        return False


def is_unique_table_name(table_name, dbt_tables):
    """
    Checks if the given string is a valid table name in PostgreSQL and dbt.

    Args:
        table_name: The string to check.
        dbt_tables: Dict of get_dbt_tables
    Returns:
        True if the string is a valid table name, False otherwise.
    """

    # The regular expression to match a valid table name.
    regex = re.compile(r"^[a-zA-Z0-9_]{1,63}$")

    # Check if the string matches the regular expression.
    if table_name not in dbt_tables:
        return True
    else:
        return False


def get_ref(original_query, dbt_tables, parsed_result, serving_tables_names):
    """
    Returns content of a user-created dbt model file w/o config.

    Args:
        original_query: Query needed processing
        dbt_tables: Dict of dicts obtained by get_tables_from_dbt.
        schema_names: List of serving schema names.

    Returns:
        ref_tables: list of models that is referenced in the query
    """
    # original_query = original_query[:-1] if original_query[-1] == ";" else original_query # Maybe unneeded since not wrapping with
    # Access table names
    fixed_query = str(original_query)
    table_names = set(get_tables_from_sql(fixed_query, dialect="postgres", sql_parsed=parsed_result))
    fixed_query = sqlfluff.fix(fixed_query, dialect="postgres")
    dbt_set = set(serving_tables_names)
    if not table_names.issubset(dbt_set):  # serving_tables_names include schema
        return None, "Tables referenced out of serving schemas"
    # Put tables in subqueries
    final_tables = tuple(table_names.intersection(serving_tables_names))  # Filter out

    if len(final_tables) == 0:
        return None, "No tables referenced in dbt projects"

    return [dbt_tables[table]["name"] for table in final_tables], "Success"


def get_records():
    # Query records
    connection = psycopg2.connect(
        user=DATABASE_USERNAME,
        password=DATABASE_PASSWORD,
        host=DATABASE_HOST,
        port=DATABASE_PORT,
        database=DATABASE_NAME,
    )
    cursor = connection.cursor()
    postgreSQL_select_Query = f"""select id, name, query_string, user_id, description, insert_time, checked, success 
                                from {QUERY_SCHEMA}.{QUERY_TABLE} where checked = False"""

    logging.info(f"Executing query to fetch records: {postgreSQL_select_Query}")
    cursor.execute(postgreSQL_select_Query)
    query_columns = [
        "id",
        "name",
        "query_string",
        "user_id",
        "description",
        "insert_time",
        "checked",
        "success",
    ]
    df = pd.DataFrame(cursor.fetchall(), columns=query_columns)

    postgreSQL_select_Query = f"select name from {QUERY_SCHEMA}.{QUERY_TABLE} where success = True"

    logging.info(f"Executing query to fetch records: {postgreSQL_select_Query}")
    cursor.execute(postgreSQL_select_Query)

    succeeded = cursor.fetchall()

    if connection:
        cursor.close()
        connection.close()
        logging.info("PostgreSQL connection is closed")
    return df, succeeded


def update_records(df):
    entries_to_update = str(tuple(zip(df.checked, df.success, df.id))).replace("None", "Null")[1:-1]
    if entries_to_update[-1]==",": entries_to_update=entries_to_update[:-1]
    connection = psycopg2.connect(
        user=DATABASE_USERNAME,
        password=DATABASE_PASSWORD,
        host=DATABASE_HOST,
        port=DATABASE_PORT,
        database=DATABASE_NAME,
    )
    cursor = connection.cursor()
    update_sql_query = f"""UPDATE {QUERY_SCHEMA}.{QUERY_TABLE} q 
                            SET success = v.success,
                                checked = v.checked

                            FROM (VALUES {entries_to_update}) AS v (checked, success, id)
                            WHERE q.id = v.id;"""
    logging.info(f"Executing query to update records: {update_sql_query}")
    cursor.execute(update_sql_query)
    connection.commit()
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        logging.info("PostgreSQL connection is closed")


def get_emails(superset, user_ids):
    url = unquote(f"/security/get_email/?q={list(user_ids)}")
    res = superset.request("GET", url)
    return res["emails"]


def get_mail_content(name, sql, status, dbt_reason=None):
    if status == "dbt success":
        message = """\
Subject: Superset Model Creation

Your Model {name} was successfully created. 

SQL:{sql}
        """.format(
            sql=sql, name=name
        )

    elif status == "dbt fail":
        message = """\
Subject: Superset Model Creation

Your Model {name} was unsuccessfully created during dbt's run, please contact the administrator.

Reason:
{reason}

SQL:
{sql}
        """.format(
            reason=dbt_reason, sql=sql, name=name
        )
    else:
        message = """\
Subject: Superset Model Creation

Your Model {name} was unsuccessfully created.

Reason:
{reason}

SQL:
{sql}
        """.format(
            reason=status, sql=sql, name=name
        )
    return message


In [2]:
status

ReloadRepositoryLocationInfo(status=<ReloadRepositoryLocationStatus.SUCCESS: 'SUCCESS'>, failure_type=None, message=None)

In [53]:
#create query

{}

In [ ]:
#pull dashboard
    superset = SupersetDBTSessionConnector()

    logging.info("Starting the script!")

    dbt = dbtRunner()
    cli_args = [
        "parse",
        "--project-dir",
        DBT_PROJECT_DIR,
    ]
    res = dbt.invoke(cli_args)
    
    
    with open('target/manifest.json') as f:
        dbt_manifest = json.load(f)

    try:
        with open(EXPOSURES_PATH) as f:
            yaml = ruamel.yaml.YAML(typ="safe")
            exposures = yaml.load(f)["exposures"]

    except (FileNotFoundError, TypeError):
        Path(EXPOSURES_PATH).parent.mkdir(parents=True, exist_ok=True)
        Path(EXPOSURES_PATH).touch(exist_ok=True)
        exposures = {}

    dbt_tables = get_tables_from_dbt(dbt_manifest, None)

    dashboards, dashboards_datasets = get_dashboards_from_superset(superset, DATABASE_ID, SUPERSET_ADMIN_ID)

    datasets = get_datasets_from_superset_dbt_refs(superset, dashboards_datasets, dbt_tables, SQL_DIALECT, DATABASE_ID)

    dashboards = merge_dashboards_with_datasets(dashboards, datasets)

    exposures_dict = get_exposures_dict(dashboards)

    # insert empty line before each exposure, except the first
    exposures_yaml = ruamel.yaml.comments.CommentedSeq(exposures_dict)  # type: ignore
    for e in range(len(exposures_yaml)):
        if e != 0:
            exposures_yaml.yaml_set_comment_before_after_key(e, before="\n")

    exposures_yaml_schema = {"version": 2, "exposures": exposures_yaml}

    exposures_yaml_file = YamlFormatted()

    with open(EXPOSURES_PATH, "w+", encoding="utf-8") as f:
        exposures_yaml_file.dump(exposures_yaml_schema, f)

    print("Transferred into a YAML file at ", EXPOSURES_PATH)
    logging.info("All done!")


In [71]:
#push description
logging.basicConfig(level=logging.INFO)

superset = SupersetDBTSessionConnector()

logging.info("Starting the script!")

sst_datasets = get_physical_datasets_from_superset(superset, DATABASE_ID)
logging.info("There are %d physical datasets in Superset.", len(sst_datasets))

if Path(MANIFEST_PATH).is_file():
    with open(MANIFEST_PATH) as f:
        dbt_manifest = json.load(f)
else:
    raise Exception("No manifest found at path")

dbt_tables = get_tables_from_dbt(dbt_manifest, None)

for i, sst_dataset in enumerate(sst_datasets):
    columns_refreshed = 0
    logging.info("Processing dataset %d/%d.", i + 1, len(sst_datasets))
    sst_dataset_id = sst_dataset["id"]
    sst_dataset_key = sst_dataset["key"]
    if sst_dataset["schema"] == USER_SCHEMA:
        continue  # Don't push user description

    refresh_columns_in_superset(superset, sst_dataset_id)
    columns_refreshed = 1

    if columns_refreshed == 1:
        columns_refreshed = 1
    else:
        refresh_columns_in_superset(superset, sst_dataset_id)
    # Otherwise, just adding the normal analytics certification
    sst_dataset_w_cols = add_superset_columns(superset, sst_dataset)
    sst_dataset_w_cols_new = merge_columns_info(sst_dataset_w_cols, dbt_tables)
    put_columns_to_superset(superset, sst_dataset_w_cols_new)
    add_sst_dataset_metadata(superset, sst_dataset_id, sst_dataset_key, dbt_tables)

logging.info("All done!")

In [ ]:
#pull description

res = superset.request("GET", f"/database/get_tables_descriptions/?db_id={DATABASE_ID}")

result = res["result"]

result_list = [
    {"name": table_dict["table_name"], "description": table_dict["table_desc"], "columns": table_dict["columns"]}
    for table_dict in result
    if table_dict["table_schema"] == USER_SCHEMA
]

for table in result_list:
    table["columns"] = [column for column in table["columns"] if column["description"]]
    if not table["columns"]:
        table.pop("columns")

desc_yaml_file = YamlFormatted()

col_desc_yaml_schema = {"version": 2, "models": result_list}
with open(DESC_YAML_PATH, "w+", encoding="utf-8") as f:
    desc_yaml_file.dump(col_desc_yaml_schema, f)

In [142]:
superset = SupersetDBTSessionConnector()
superset_tables_dict_list = get_physical_datasets_from_superset(superset, DATABASE_ID)
superset_tables_id_dict = dict([(table["key"], table["id"]) for table in superset_tables_dict_list])
logging.info("Starting!")

dbt = dbtRunner()
cli_args = [
    "parse",
    "--project-dir",
    DBT_PROJECT_DIR,
]
res = dbt.invoke(cli_args)

dbt_tables = {}
with open('target/manifest.json') as f:
    dbt_manifest = json.load(f)
dbt_tables_temp = get_tables_from_dbt(dbt_manifest, None)
dbt_tables = {**dbt_tables, **dbt_tables_temp}

    

# Getting the dbt tables keys
dbt_tables_names = list(dbt_tables.keys())
logging.info("Tables in manifest: {num}".format(num=len(dbt_tables_names)))
superset_dict_keys = [i["key"] for i in superset_tables_dict_list]
# Only tables that start with a given schema prefix name
mapped = map(lambda x: x.startswith((SERVING_SCHEMA, USER_SCHEMA)), dbt_tables_names)
mask = list(mapped)

dbt_tables_reporting = list(compress(dbt_tables_names, mask))
superset_tables = superset_dict_keys

# Parsing as sets
dbt_tables_reporting = set(dbt_tables_reporting)
superset_tables = set(superset_tables)

# To add to superset
add_to_superset = list(dbt_tables_reporting.difference(superset_tables))

# To remove from superset

remove_from_superset = list(superset_tables.difference(dbt_tables_reporting))

for i in add_to_superset:
    logging.info("Starting datasets addition")
    rison_request = "dataset/"
    array = i.split(".")
    schema = array[0]
    table_name = array[1]
    # Data to be written
    dictionary = {
        # Parameter database
        "database": DATABASE_ID,
        "schema": schema,
        "table_name": array[1],
        "owners": [SUPERSET_ID],
    }
    # Add potential user
    if dbt_tables[i]["user"]:
        dictionary["owners"].append(int(dbt_tables[i]["user"]))
    # Serializing json
    json_object = json.dumps(dictionary)
    try:
        response = superset.request("POST", rison_request, json=dictionary)
    except:
        raise Exception(dictionary)
logging.info("Done!")
logging.info("Starting superset datasets removal")
for i in remove_from_superset:
    # Dataset id to be deleted
    dataset_id = superset_tables_id_dict[i]

    rison_request = "/dataset/" + str(dataset_id)
    response = superset.request("DELETE", rison_request)
logging.info("Done with removing tables!")


DEBUG:file_log:12:58:17.445296 [debug] [MainThread]: Sending event: {'category': 'dbt', 'action': 'invocation', 'label': 'start', 'context': [<snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fc529e680>, <snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fc529ea40>, <snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fc529e770>]}


05:58:17  Running with dbt=1.5.1


INFO:stdout_log:05:58:17  Running with dbt=1.5.1
INFO:file_log:

============================== 12:58:17.447549 | cde03b94-b823-4cdd-a0f9-3e9e8255a356 ==============================
12:58:17.447549 [info ] [MainThread]: Running with dbt=1.5.1
DEBUG:file_log:12:58:17.452186 [debug] [MainThread]: running dbt with arguments {'printer_width': '80', 'indirect_selection': 'eager', 'write_json': 'True', 'log_cache_events': 'False', 'partial_parse': 'True', 'cache_selected_only': 'False', 'warn_error': 'None', 'version_check': 'True', 'profiles_dir': '/home/vu/.dbt', 'log_path': '/home/vu/Desktop/Projects/Thesis/financial/dbt/logs', 'fail_fast': 'False', 'debug': 'False', 'use_colors': 'True', 'use_experimental_parser': 'False', 'no_print': 'None', 'quiet': 'False', 'log_format': 'default', 'static_parser': 'True', 'warn_error_options': 'WarnErrorOptions(include=[], exclude=[])', 'introspect': 'True', 'target_path': 'None', 'send_anonymous_usage_stats': 'True'}
DEBUG:file_log:12:58:17.490912 [

05:58:17  Performance info: target/perf_info.json


INFO:stdout_log:05:58:17  Performance info: target/perf_info.json
INFO:file_log:12:58:17.819912 [info ] [MainThread]: Performance info: target/perf_info.json
DEBUG:file_log:12:58:17.874199 [debug] [MainThread]: Command `cli parse` succeeded at 12:58:17.873816 after 0.43 seconds
DEBUG:file_log:12:58:17.876794 [debug] [MainThread]: Sending event: {'category': 'dbt', 'action': 'invocation', 'label': 'end', 'context': [<snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fbc8ebb80>, <snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fc529e860>, <snowplow_tracker.self_describing_json.SelfDescribingJson object at 0x7f8fbc70be80>]}
DEBUG:file_log:12:58:17.879711 [debug] [MainThread]: Flushing usage events


In [133]:
add_to_superset

['marts.fact_business_model_rating',
 'marts.fact_valuation_rating',
 'marts.fact_bollinger',
 'marts.fact_general_rating',
 'marts.fact_income_statement',
 'marts.fact_cash_flow',
 'marts.fact_balance_sheet',
 'user.user_created_example',
 'marts.fact_bov',
 'marts.fact_mfi',
 'marts.fact_financial_health_rating',
 'marts.fact_business_operation_rating',
 'marts.fact_industry_health_rating']

In [132]:
dbt_tables[add_to_superset]

TypeError: unhashable type: 'list'

In [127]:
dbt_tables[i]["user"]

'1'

In [130]:
dictionary

{'database': 1,
 'schema': 'user',
 'table_name': 'user_created_example',
 'owners': [36, '1']}

In [129]:
dictionary["owners"].append(dbt_tables[i]["user"])

In [107]:
response = superset.request("POST", rison_request, json=dictionary)

HTTPError: 422 Client Error: UNPROCESSABLE ENTITY for url: http://34.82.185.252:30007/api/v1/dataset/

In [108]:
response

<Response [200]>

In [103]:
superset = SupersetDBTSessionConnector()
superset.request("GET", "security/csrf_token/")

{'result': 'Ijc0YzhmMGQzNDg0MjRjNDhlY2ZmN2NiMTg0NjgxOWI2M2IzOTdjNzQi.ZNcW_g.jIT6WcA2XrpbD_VyUGrnl9YX0-Y'}

In [98]:
SUPERSET_USERNAME

'superset'

In [102]:

session = requests.session()
soup = BeautifulSoup(session.post(SUPERSET_HOST + "login").text, "html.parser")
csrf_token = soup.find("input", {"id": "csrf_token"})["value"]  # type: ignore

data = {
    "username": "fdp",
    "password": "fdp",
    "provider": "db",
    "refresh": True,
}
headers = {
    # 'Authorization': 'Bearer {}'.format(self.____access_token),
    "x-csrftoken": csrf_token,
}
response = session.post(SUPERSET_HOST + "login", json=data, headers=headers)  # type: ignore

endpoint ="security/csrf_token/"
method = "GET"
api_url = SUPERSET_HOST +"api/v1/"

logging.info("About to %s execute request for endpoint %s", method, endpoint)

url = api_url + endpoint
csrf_headers = {
    # 'Authorization': 'Bearer {}'.format(self.__access_token),
    "x-csrftoken": csrf_token,
}

res = session.request(method, url, headers=csrf_headers)  # type: ignore

logging.info("Request finished with status: %d", res.status_code)

# if res.status_code == 401 and res.json().get("msg") == "Token has expired" and self.__refresh_session():
#     logging.info(f"Retrying {method} request for {url} %s with refreshed session")
#     res = self.__session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

#     logging.info("Request finished with status: %d", res.status_code)

# if (
#     res.status_code == 400
#     and res.json()["message"] == "400 Bad Request: The CSRF session token is missing."
#     and self.__refresh_session()
# ):
#     logging.info(f"Retrying {method} request for {url} %s with refreshed session")
#     res = self.__session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore
#     logging.info(f"Request finished with status: {res.status_code}")
res.raise_for_status()

In [94]:
csrf_token

'IjM2ODljZDkwYWJkMjk5ODk3YzQ1NTY3YjEwZmQ3NGNkYmIzODhlM2Ei.ZNcWlA.wpDLbRG9SDj5cIhdpO739ooCbhM'

In [95]:
res.text

'{"result":"IjM2ODljZDkwYWJkMjk5ODk3YzQ1NTY3YjEwZmQ3NGNkYmIzODhlM2Ei.ZNcWlg.Jj2oIbjzHgq1I5kf1LdprrFL7ow"}\n'

In [26]:
session

In [101]:
SUPERSET_USERNAME

'fdp'

In [80]:
import requests
# from bs4 import BeautifulSoup

username = "admin"
password = "admin"
session = requests.session()

login_form = session.post(SUPERSET_HOST+'/login')
soup = BeautifulSoup(login_form.text, 'html.parser')
csrf_token = soup.find('input',{'id':'csrf_token'})['value']
data = {
  'username': username,
  'password': password,
  # 'csrf_token': csrf_token
}
response = session.post(SUPERSET_HOST+'/login', data=data)
# response = session.get('http://localhost:8088/api/v1/me')

In [82]:
csrf_token

'ImI2MjU0MDAxOTczNDYxZmQ4ZTUyMmNjMjMwODFiNTkzZWJiMTk0Yjki.ZNcWKA.pn4FcmWsPWXGmSb93zjQJAQ_TSc'

In [81]:
response

<Response [200]>

In [104]:
superset = SupersetDBTSessionConnector()
logging.info("Getting physical datasets from Superset.")
page_number = 0
datasets = []
datasets_keys = set()
while True:
    logging.info("Getting page %d.", page_number + 1)
    rison_request = f"dataset/?q=(page_size:100,page:{page_number},order_column:changed_on_delta_humanized,order_direction:asc,filters:!((col:table_name,opr:nct,value:archived),(col:sql,opr:dataset_is_null_or_empty,value:true)))"
    res = superset.request("GET", rison_request)
    result = res["result"]
    if result:
        for r in result:
            name = r["table_name"]
            schema = r["schema"]
            database_name = r["database"]["database_name"]
            dataset_id = r["id"]
            database_id = r["database"]["id"]
            dataset_key = f"{schema}.{name}"  # same format as in dashboards

            kind = r["kind"]
            if kind == "physical" and (1 is None or database_id == 1):
                dataset_id = r["id"]

                name = r["table_name"]
                schema = r["schema"]
                dataset_key = f"{schema}.{name}"  # used as unique identifier

                dataset_dict = {
                    "id": dataset_id,
                    "name": name,
                    "schema": schema,
                    "database": database_name,
                    "dataset_id": dataset_id,
                    "key": dataset_key,
                    "table": [dataset_key],
                }

                # fail if it breaks uniqueness constraint
                assert dataset_key not in datasets_keys, (
                    f"Dataset {dataset_key} is a duplicate name (schema + table) "
                    "across databases. "
                    "This would result in incorrect matching between Superset and dbt. "
                    "To fix this, remove duplicates or add the ``superset_db_id`` argument."
                )

                datasets_keys.add(dataset_key)
                datasets.append(dataset_dict)

        page_number += 1
    else:
        break

In [124]:
class SupersetDBTSessionConnector:
    """A class for accessing the Superset API in an easy way."""

    def __init__(self):
        """Instantiates the class.

        ''access_token'' will be instantiated via enviromental variable
        If ``access_token`` is None, attempts to obtain it using ``refresh_token``.

        Args:
            api_url: Base API URL of a Superset instance, e.g. https://my-superset/api/v1.
            access_token: Access token to use for accessing protected endpoints of the Superset
                API. Can be automatically obtained if ``refresh_token`` is not None.
            refresh_token: Refresh token to use for obtaining or refreshing the ``access_token``.
                If None, no refresh will be done.
        """
        self.url = SUPERSET_HOST
        self.api_url = self.url + "api/v1/"

        self.session = requests.session()

        self.username = SUPERSET_USERNAME
        self.password = SUPERSET_PASSWORD

        self.refresh_session()

    def refresh_session(self):
        logging.info("Refreshing session")

        soup = BeautifulSoup(self.session.post(self.url + "login").text, "html.parser")
        self.csrf_token = soup.find("input", {"id": "csrf_token"})["value"]  # type: ignore

        data = {
            "username": self.username,
            "password": self.password,
            "provider": "db",
            "refresh": True,
        }
        headers = {
            # 'Authorization': 'Bearer {}'.format(self.____access_token),
            "x-csrftoken": self.csrf_token,
        }
        response = self.session.post(self.url + "login", json=data, headers=headers)  # type: ignore
        return True

    def request(self, method, endpoint, **request_kwargs):
        """Executes a request against the Superset API.

        Args:
            method: HTTP method to use.
            endpoint: Endpoint to use.
            **request_kwargs: Any ``requests.request`` arguments to use.

        Returns:
            A dictionary containing response body parsed from JSON.

        Raises:
            HTTPError: There is an HTTP error (detected by ``requests.Response.raise_for_status``)
                even after retrying with a fresh session.
        """

        logging.info("About to %s execute request for endpoint %s", method, endpoint)

        url = self.api_url + endpoint
        csrf_headers = {
            # 'Authorization': 'Bearer {}'.format(self.__access_token),
            "x-csrftoken": self.csrf_token,
        }

        res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

        logging.info("Request finished with status: %d", res.status_code)

        # if res.status_code and self.refresh_session():
        #     logging.info(f"Retrying {method} request for {url} %s with refreshed session")
        #     res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

        #     logging.info("Request finished with status: %d", res.status_code)

        # if (
        #     res.status_code == 400
        #     and self.refresh_session()
        # ):
        #     logging.info(f"Retrying {method} request for {url} %s with refreshed session")
        #     res = self.session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore
        #     logging.info(f"Request finished with status: {res.status_code}")
        res.raise_for_status()
        return res.json()

In [112]:
superset=SupersetDBTSessionConnector()

In [122]:
superset.csrf_token

'ImUwYjk5NWE1YmMxYjFjMGNlNzg5YWYyZGFmNWYwOGQ0ZjJkMmY3ZmUi.ZNcbCw.bHkXfs6xKdFJnqpySvRWzRqszNs'

In [120]:
superset = SupersetDBTSessionConnector()
superset.request("GET", "database/get_tables_descriptions/?db_id=1")

{'result': [{'columns': [{'description': None, 'name': 'ticker'},
    {'description': None, 'name': 'website'},
    {'description': None, 'name': 'outstanding_share'},
    {'description': None, 'name': 'company_type'},
    {'description': None, 'name': 'delta_in_year'},
    {'description': None, 'name': 'industry'},
    {'description': None, 'name': 'stock_rating'},
    {'description': None, 'name': 'industry_en'},
    {'description': None, 'name': 'no_employees'},
    {'description': None, 'name': 'delta_in_week'},
    {'description': None, 'name': 'industry_id'},
    {'description': None, 'name': 'industry_id_v2'},
    {'description': None, 'name': 'exchange'},
    {'description': None, 'name': 'established_year'},
    {'description': None, 'name': 'short_name_en'},
    {'description': None, 'name': 'no_share_holders'},
    {'description': None, 'name': 'issue_share'},
    {'description': None, 'name': 'delta_in_month'},
    {'description': None, 'name': 'foreign_percent'}],
   'tabl

In [121]:
superset.request("GET", "database/get_tables_descriptions/?db_id=1")

{'result': [{'columns': [{'description': None, 'name': 'ticker'},
    {'description': None, 'name': 'website'},
    {'description': None, 'name': 'outstanding_share'},
    {'description': None, 'name': 'company_type'},
    {'description': None, 'name': 'delta_in_year'},
    {'description': None, 'name': 'industry'},
    {'description': None, 'name': 'stock_rating'},
    {'description': None, 'name': 'industry_en'},
    {'description': None, 'name': 'no_employees'},
    {'description': None, 'name': 'delta_in_week'},
    {'description': None, 'name': 'industry_id'},
    {'description': None, 'name': 'industry_id_v2'},
    {'description': None, 'name': 'exchange'},
    {'description': None, 'name': 'established_year'},
    {'description': None, 'name': 'short_name_en'},
    {'description': None, 'name': 'no_share_holders'},
    {'description': None, 'name': 'issue_share'},
    {'description': None, 'name': 'delta_in_month'},
    {'description': None, 'name': 'foreign_percent'}],
   'tabl